In [ ]:
!pip3 install transformers
pip3 install transformers==2.9.0

In [ ]:
from transformers import AutoTokenizer, AutoModel
from transformers import BartForSequenceClassification, BartTokenizer
from torch.nn import functional as F
import tensorflow as tf
import operator
import pprint
import torch
import os

In [ ]:
from ipywidgets import IntProgress

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('deepset/sentence_bert')
model = AutoModel.from_pretrained('deepset/sentence_bert')

In [ ]:
!ls

In [ ]:
labels = ['unhappy', 'happy', 
           'positive', 'negative', 'neutral']

sentence = '''
Hi, 
I had very good experience in using the service.
I have recently made use of Virtual Shopping to help me to rearrang home delivery for my order.
Because I want to change the delivery date, can you share the instructions.
Kindly regards,
Lu
'''
[sentence] + labels

In [ ]:
# Encode and tokenize the sentence and labels
inputs = tokenizer.batch_encode_plus([sentence] + labels,
                                     return_tensors='pt',
                                     pad_to_max_length=True)
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']

In [ ]:
response = 'Sentence submitted for labelling \n' + str(sentence)+'\n'

output = model(input_ids, attention_mask=attention_mask)[0]
sentence_emb = output[:1].mean(dim=1)
label_emb = output[1:].mean(dim=1)

print(output.shape)
print(sentence_emb.shape)
print(label_emb.shape)
    
similarities = F.cosine_similarity(sentence_emb, label_emb)
closest = similarities.argsort(descending=True)

result = dict(zip(labels, similarities.tolist()))
sorted_d = sorted(result.items(), key=operator.itemgetter(1), reverse=True)
pprint.pprint(sorted_d)


print('Assigned Labels')
for ind in closest:
    response += f'label: {labels[ind]}    \t similarity: {round(similarities[ind].item(), 2)} \n'
print(response)

In [ ]:
print(similarities.tolist())
print(closest)

In [ ]:
!pwd

In [ ]:
LOCAL_MODEL_DIR = "/Users/napt/Workspace/gft/model-serving-examples/aiplatform/model_files/model.pt"
torch.save(model, LOCAL_MODEL_DIR)

In [ ]:
!ls -ltrh model_files

In [ ]:
!ls model_files

In [ ]:
model_dir = '/Users/napt/Workspace/gft/model-serving-examples/aiplatform/model_files'

In [ ]:
model_file = os.path.join(model_dir, 'model.pt')
print(model_file)
model = torch.load(model_file)

In [ ]:
output = model(input_ids, attention_mask=attention_mask)[0]

In [ ]:
output

In [ ]:
import os
import pandas as pd
from google.cloud import storage
import torch
from transformers import AutoTokenizer, AutoModel
from transformers import BartForSequenceClassification, BartTokenizer
from torch.nn import functional as F
import os


class EmailClassifier(object):
    def __init__(self, model):
        self._model = model
        self.class_lables = ['unhappy', 'happy', 'positive', 'negative', 'neutral']

    @classmethod
    def from_path(cls, model_dir):
        model_file = os.path.join(model_dir, 'model.pt')
        model = torch.load(model_file)
        return cls(model)

    def predict(self, sentence, **kwargs):
        inputs = tokenizer.batch_encode_plus([sentence] + self.class_lables,
                                             return_tensors='pt',
                                             pad_to_max_length=True)
        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']
        output = self._model(input_ids, attention_mask=attention_mask)[0]
        sentence_emb = output[:1].mean(dim=1)
        label_emb = output[1:].mean(dim=1)

        similarities = F.cosine_similarity(sentence_emb, label_emb)

        result = dict(zip(self.class_lables, similarities.tolist()))
        predicted = sorted(result.items(), key=operator.itemgetter(1),
                           reverse=True)

        return predicted

In [ ]:
model = EmailClassifier.from_path(model_dir)
eclass = EmailClassifier(model)

In [ ]:
eclass.model(input_ids, attention_mask=attention_mask)

In [ ]:
model_file = os.path.join(model_dir, 'model.pt')
model = torch.load(model_file)

In [ ]:
inputs = tokenizer.batch_encode_plus([sentence] + labels,
                                             return_tensors='pt',
                                             pad_to_max_length=True)
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']
output = model(input_ids, attention_mask=attention_mask)[0]
sentence_emb = output[:1].mean(dim=1)
label_emb = output[1:].mean(dim=1)

similarities = F.cosine_similarity(sentence_emb, label_emb)

result = dict(zip(labels, similarities.tolist()))
predicted = sorted(result.items(), key=operator.itemgetter(1),reverse=True)

In [ ]:
predicted